# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Spark SQL** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Find the PySpark Installation

In [ ]:
import findspark
findspark.init()

# Create SparkSession

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://a8d253fcd1c5:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [ ]:
from pcamarillor.spark_utils import SparkUtils
schema = SparkUtils.generate_schema([("name", "string"), ("age", "int"), ("city", "string")])

## Example: Smart Factory Sensor Data

In [ ]:
from datetime import datetime

factory_data = [
    ("M001", datetime(2025, 4, 26, 8, 0, 0), 75.3),
    ("M002", datetime(2025, 4, 26, 8, 5, 0), 68.7),
    ("M001", datetime(2025, 4, 26, 8, 10, 0), 76.1),
    ("M003", datetime(2025, 4, 26, 8, 15, 0), 72.4),
    ("M002", datetime(2025, 4, 26, 8, 20, 0), 69.8),
    ("M001", datetime(2025, 4, 26, 8, 25, 0), 77.5),
    ("M003", datetime(2025, 4, 26, 8, 30, 0), 73.2),
    ("M002", datetime(2025, 4, 26, 8, 35, 0), 70.1),
    ("M001", datetime(2025, 4, 26, 8, 40, 0), 78.0),
    ("M003", datetime(2025, 4, 26, 8, 45, 0), 74.6),
]

factory_schema = SparkUtils.generate_schema([("machine_id", "string"), ("sensor_timestamp", "timestamp"), ("temp", "float")])
df_factory = spark.createDataFrame(factory_data, factory_schema)
df_factory.show()

In [ ]:
sc.stop()